In [39]:
import json, requests, csv
import pandas as pd
import csv, re, time
from datetime import datetime
from textblob import TextBlob
import numpy as np
from bs4 import BeautifulSoup
from dateutil.parser import parse
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
import math

In [40]:
# creation du fichier company

url_company = 'C:/Users/chris/OneDrive/Bureau/PrescriptiveFS/webscrap/03032024/Company-2024-03-03.csv'
data_company = pd.read_csv(url_company)


# creation du fichier keyword

url_GoogleSearchKeyword = 'C:/Users/chris/OneDrive/Bureau/PrescriptiveFS/webscrap/03032024/GoogleSearchKeyword-2024-03-03.csv'
data_GoogleSearchKeyword = pd.read_csv(url_GoogleSearchKeyword)

# creation du fichier googleresults

url_google_results = 'C:/Users/chris/OneDrive/Bureau/PrescriptiveFS/webscrap/03032024/GoogleSearchResult-2024-03-03.csv'
data_google_results = pd.read_csv(url_google_results)

In [41]:
data_company.head()

,id,siren,name,brand_name,description,address,city,postal_code,country,phone,...,ape,founded_date,turnover_2022,gouv_searched,gouv_street,gouv_city,gouv_postal_code,gouv_country,gouv_lat,gouv_lng
0,164,847593225,AAA COIFFURE,AAA COIFFURE,NaN,11 RUE PAGANINI,NICE,6000,NaN,NaN,...,9602A,2019-01-15,33300.0,1.0,Rue Paganini,Nice,6000.0,NaN,43.702199,7.263926
1,203,331057406,ACA NEXIA,ACA NEXIA,NaN,31 RUE HENRI ROCHEFORT,PARIS,75017,NaN,NaN,...,6920Z,1984-09-01,18602980.0,1.0,Rue Henri Rochefort,Paris,75017.0,NaN,48.882511,2.308276
2,163,845298249,AEV AUTO ECOLE VEILLON,AUTO ECOLE VEILLON,NaN,11 AVENUE VILLERMONT,NICE,6000,NaN,NaN,...,8553Z,2019-01-09,244188.0,1.0,Avenue Villermont,Nice,6000.0,NaN,43.709300,7.264297
3,527,316473305,AGRI-POLE,AGRI-POLE,NaN,RUE SAINT VICTOR,VILLEFRANCHE-DE-ROUERGUE,34420,NaN,NaN,...,4661Z,1979-01-01,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,310,349638395,AIR CORSICA,CCM AIRLINES,NaN,CAMPO DELL ORO,AJACCIO,20090,NaN,NaN,...,5110Z,1990-01-01,130850508.0,1.0,Campo Dell Oro,Ajaccio,20090.0,NaN,41.929606,8.804378


In [42]:
data_company[data_company['id'] == 320]

,id,siren,name,brand_name,description,address,city,postal_code,country,phone,...,ape,founded_date,turnover_2022,gouv_searched,gouv_street,gouv_city,gouv_postal_code,gouv_country,gouv_lat,gouv_lng
268,320,377550249,WAVESTONE,WAVESTONE,NaN,100 TERRASSE BOIELDIEU,PUTEAUX,92800,NaN,NaN,...,6202A,1990-04-01,377647000.0,1.0,Terrasse Boieldieu,Puteaux,92800.0,NaN,48.889287,2.24075


In [43]:
data_GoogleSearchKeyword.head()

,id,active,keyword,news
0,116,1,"""actualités""",0
1,149,1,"""annonce""",0
2,154,1,"""controverses"" OR ""problème"" OR ""litige""",0
3,153,1,"""événement"" OR ""séminaire"" OR ""atelier""",0
4,151,1,"""lancement de produit"" OR ""nouveau service""",0


In [44]:
data_google_results.head()

,id,search,company,google_results,scraped_date,scraped
0,1999,116,545,NaN,2024-03-03 07:02:38,NaN
1,372,116,309,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:39:31,1.0
2,371,116,491,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:39:27,1.0
3,370,116,506,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:39:23,1.0
4,369,116,320,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:39:24,1.0


In [45]:
company_list = []
SIRET_list = []

# Itérer sur chaque ligne du DataFrame
for index, occurrence in data_company.iterrows():
    company = occurrence['id']  # Assurez-vous que 'id' est le nom correct de la colonne
    SIRET = occurrence['siret']  # Assurez-vous que 'siret' est le nom correct de la colonne
    company_list.append(company)
    SIRET_list.append(SIRET)

Id_SIRET = dict(zip(company_list, SIRET_list))

In [46]:
Pk_search_list = []
keyword_list = []

# Itérer sur chaque ligne du DataFrame
for index, occurrence in data_GoogleSearchKeyword.iterrows():
    Pk_search = occurrence['id']
    keyword = occurrence['keyword']
    Pk_search_list.append(Pk_search)
    keyword_list.append(keyword)

Search_keyword = dict(zip(Pk_search_list, keyword_list))

In [47]:
Search_keyword

{116: '"actualités"',
 149: '"annonce"',
 154: '"controverses" OR "problème" OR "litige"',
 153: '"événement" OR "séminaire" OR "atelier"',
 151: '"lancement de produit" OR "nouveau service"',
 152: '"nomination" OR "départ" OR "nouveau dirigeant"',
 150: '"partenariat" OR "collaboration"'}

In [48]:
data_modifie = []

# Parcourir chaque élément du google search, créer un dictionnaire id (company) / SIRET / keyword / contenu

for index, element in data_google_results.iterrows():
    nouvel_element = {}
    for cle, valeur in element.items():
        if cle == 'company':
            nouvel_element['SIRET'] = Id_SIRET[element['company']] # affecter le SIRET en fonction de la table company (Id)
        elif cle == 'search':
            nouvel_element['keyword'] = Search_keyword[element['search']]
        else:
            nouvel_element[cle] = valeur
    data_modifie.append(nouvel_element)

In [49]:
data_modifie = pd.DataFrame(data_modifie)

In [50]:
data_modifie

,id,keyword,SIRET,google_results,scraped_date,scraped
0,1999,"""actualités""",408627354 00124,NaN,2024-03-03 07:02:38,NaN
1,372,"""actualités""",34899155500353,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:39:31,1.0
2,371,"""actualités""",30649449300050,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:39:27,1.0
3,370,"""actualités""",31124863700804,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:39:23,1.0
4,369,"""actualités""",37755024900041,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:39:24,1.0
...,...,...,...,...,...,...
1899,1732,"""partenariat"" OR ""collaboration""",34963839500021,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:56:31,1.0
1900,1731,"""partenariat"" OR ""collaboration""",31647330500035,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:56:30,1.0
1901,1730,"""partenariat"" OR ""collaboration""",84529824900028,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:56:47,1.0
1902,1729,"""partenariat"" OR ""collaboration""",33105740600067,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:56:37,1.0


In [51]:
def extraire_donnees(json_str, siret, keyword):
    try:
        # Assurez-vous que json_str est une chaîne
        if not isinstance(json_str, str):
            print(f"Erreur : json_str n'est pas une chaîne. SIRET: {siret}, Keyword: {keyword}")
            return []

        # Charger la chaîne JSON en dictionnaire
        data = json.loads(json_str)
        results = []
        
        # Itération sur les résultats organiques
        for result in data.get('organic_results', []):
            res_dict = {
                'siret': siret,
                'keyword': keyword,
                'url': result.get('url', ''),
                'title': result.get('title', ''),
                'domain': result.get('domain', ''),
                'description': result.get('description', ''),
                'displayed_url': result.get('displayed_url', '')
            }
            if isinstance(result.get('sitelinks'), dict):
                res_dict['sitelinks'] = result.get('sitelinks', {}).get('expanded', [])
            else:
                res_dict['sitelinks'] = []
            results.append(res_dict)
        
        return results
    except Exception as e:
        print(f"Erreur lors de l'extraction: {e}, siret: {siret}, keyword: {keyword}")
        return []
    
resultats_extraits = []

# Itérer sur chaque ligne du DataFrame data_modifie
for idx, row in data_modifie.iterrows():
    # Extraire les données organiques, le siret et le mot-clé pour chaque ligne
    resultats_temp = extraire_donnees(row['google_results'], row['SIRET'], row['keyword'])
    # Ajouter les résultats à la liste principale
    resultats_extraits.extend(resultats_temp)

# Convertir la liste des résultats extraits en un nouveau DataFrame
detail_google_search = pd.DataFrame(resultats_extraits)

Erreur : json_str n'est pas une chaîne. SIRET: 408627354 00124, Keyword: "actualités"
Erreur : json_str n'est pas une chaîne. SIRET: 408627354 00124, Keyword: "annonce"
Erreur : json_str n'est pas une chaîne. SIRET: 408627354 00124, Keyword: "controverses" OR "problème" OR "litige"
Erreur : json_str n'est pas une chaîne. SIRET: 408627354 00124, Keyword: "événement" OR "séminaire" OR "atelier"
Erreur : json_str n'est pas une chaîne. SIRET: 408627354 00124, Keyword: "lancement de produit" OR "nouveau service"
Erreur : json_str n'est pas une chaîne. SIRET: 408627354 00124, Keyword: "nomination" OR "départ" OR "nouveau dirigeant"
Erreur : json_str n'est pas une chaîne. SIRET: 408627354 00124, Keyword: "partenariat" OR "collaboration"


In [52]:
detail_google_search

,siret,keyword,url,title,domain,description,displayed_url,sitelinks
0,34899155500353,"""actualités""",https://www.nicematin.com/conso-shopping/le-pl...,Le plus grand Zara de la région Sud a ouvert c...,www.nicematin.com,"May 4, 2023 — Nice-Matin. Se connecter · Abonn...",https://www.nicematin.com › le-plu...,[]
1,34899155500353,"""actualités""",https://nicepresse.com/le-plus-grand-zara-de-t...,Le plus grand Zara de tout le Sud-Est bientôt ...,nicepresse.com,"Oct 31, 2022 — Nice-Presse : actualités et inf...",https://nicepresse.com › le-plus-gra...,[]
2,34899155500353,"""actualités""",https://actu.fr/provence-alpes-cote-d-azur/nic...,Nice. Pourquoi y avait-il une grosse file d'at...,actu.fr,"Nov 25, 2022 — Dernières actualités. Actu Nice...",https://actu.fr › nice_06088 › nice-...,[]
3,34899155500353,"""actualités""",https://actu.fr/provence-alpes-cote-d-azur/sai...,"Près de Nice, CAP 3000 annonce de nouvelles bo...",actu.fr,"Jan 19, 2023 — L'un des plus grands Zara de Fr...",https://actu.fr › pres-de-nice-cap-30...,[]
4,34899155500353,"""actualités""",https://www.lefigaro.fr/tag/zara,Zara : dernières actualités et vidéos,www.lefigaro.fr,Les dernières actualités sur Zara.,https://www.lefigaro.fr › tag › zara,[]
...,...,...,...,...,...,...,...,...
13930,84759322500018,"""partenariat"" OR ""collaboration""",https://books.google.fr/books?id=9QIdYbTIKQAC&...,Le Mémorial diplomatique,books.google.fr,... AAA Vient de Paraitre chez PAUL OLLENDORFF...,https://books.google.fr › books,[]
13931,84759322500018,"""partenariat"" OR ""collaboration""",https://books.google.fr/books?id=PbKXBAAAQBAJ&...,Michelin Green Guide France - Google Books Result,books.google.fr,... collaboration with Coysevox and Tuby. In t...,https://books.google.fr › books,[{'link': '/search?num=100&sca_esv=566872717&g...
13932,84759322500018,"""partenariat"" OR ""collaboration""",https://books.google.fr/books?id=CjiXEAAAQBAJ&...,Sekiro: La seconde vie des Souls - Google Book...,books.google.fr,... partenariat avec Activision en surprend – ...,https://books.google.fr › books,[{'link': '/search?num=100&sca_esv=566872717&g...
13933,84759322500018,"""partenariat"" OR ""collaboration""",https://books.google.fr/books?id=qZZ-CwAAQBAJ&...,Robert Trent Jones Golf Trail: Its History and...,books.google.fr,... collaboration between the City of Montgome...,https://books.google.fr › books,[{'link': '/search?num=100&sca_esv=566872717&g...


In [53]:
# Obtenir la date actuelle
date_actuelle = datetime.now().strftime('%Y-%m-%d')  # Formate la date comme "AAAA-MM-JJ"

# Créer le nom du fichier avec la date
nom_fichier = f'search_Google_{date_actuelle}.csv'

# Exporter le DataFrame en CSV
detail_google_search.to_csv(nom_fichier, index=False)

In [54]:
# URL de la page à scraper
url = 'https://www.nicematin.com/conso-shopping/le-plus-grand-zara-de-la-region-sud-a-ouvert-ce-jeudi-a-cap-3000-avec-un-concept-dernier-cri-845377'

# Effectuer une requête GET sur le site et Créer un objet BeautifulSoup pour analyser le HTML
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
text = ' '.join(p.text for p in soup.find_all('p'))
meta_date = soup.find('meta', {'property': 'article:published_time'})

# Extraire le texte souhaité, par exemple, tous les paragraphes et  Créer un résumé du texte
parser = PlaintextParser.from_string(text, Tokenizer('french'))
summarizer = Summarizer()

# Imprimer les points les plus importants du texte
for sentence in summarizer(parser.document, 30):  # Modifier le '5' pour changer le nombre de phrases du résumé
    print(sentence)
    
    

Lire le journal Lire le journal Se connecter L’enseigne avait fermé ses portes il y a six mois.
Zara a signé son grand retour ce jeudi après-midi, dans le centre commercial Cap 3000, à Saint-Laurent-du-Var.
Le magasin laurentin devient le plus grand Zara de la région Sud.
Si l’emplacement n’a pas changé, la boutique est passée de 900 à 3.000mètres carrés, sur un étage.
En plus de proposer de l’habillement femme, elle dispose désormais d’un coin homme et d’un coin enfant.
Son retour était attendu par les clientes.
En début d’après-midi, devant la grille de fer qui s’apprêtait à ouvrir, Mélanie trépignait d’impatience.
"Je n’avais pas le temps d’aller dans le Zara d’une autre ville, j’habite à Saint-Laurent.
Il nous a manqué, surtout pendant les fêtes.
Ça fait six mois que je ne me suis pas acheté du Zara, je n’en peux plus!
", a-t-elle confié, grand sourire.
Les clientes étaient d’autant plus ravies que le concept du magasin est du dernier cri.
Via l’application, vous pouvez désormais r

In [55]:

if meta_date:
    print(meta_date.get('content'))

2023-05-04T15:25:00+02:00


In [56]:
# Ajoutez des colonnes pour le résumé et la date dans le DataFrame
detail_google_search['resume'] = pd.NA
detail_google_search['date'] = pd.NA

In [57]:
def scraper_et_resume(url):
    attempts = 0
    while attempts < 2:  # Deux tentatives
        try:
            # Effectuer une requête GET sur le site
            response = requests.get(url, timeout=10)  # Augmentez le timeout si nécessaire
            # Créer un objet BeautifulSoup pour analyser le HTML
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extraire le texte souhaité, par exemple, tous les paragraphes
            text = ' '.join(p.text for p in soup.find_all('p'))
            # Recherche de la date dans les métadonnées
            meta_date = soup.find('meta', {'property': 'article:published_time'})
            date = meta_date['content'] if meta_date else 'Date non trouvée'

            # Création d'un résumé du texte
            parser = PlaintextParser.from_string(text, Tokenizer('french'))
            summarizer = Summarizer()
            resume_text = ' '.join(str(sentence) for sentence in summarizer(parser.document, 5))  # Modifier le '5' pour changer le nombre de phrases du résumé

            return resume_text, date
        except requests.exceptions.RequestException as e:  # Attrape toutes les exceptions liées aux requêtes
            print(f"Erreur lors de l'accès à {url}: {e}")
            attempts += 1
            time.sleep(5)  # Délai entre les tentatives
    
    # Retourner None si toutes les tentatives échouent
    return None, None

In [ ]:

# Calculer le total des lignes et 10% de ce total pour les points de contrôle
total_lignes = len(detail_google_search)
dix_pourcent = total_lignes / 10

# Initialiser une variable pour suivre la progression
progression = 0

for index, row in detail_google_search.iterrows():
    try:
        resume, date = scraper_et_resume(row['url'])
        detail_google_search.at[index, 'resume'] = resume
        detail_google_search.at[index, 'date'] = date
    except Exception as e:  # Attraper n'importe quelle exception survenue pendant le scraping et le résumé
        print(f"Erreur lors du traitement de la ligne {index}: {e}")
        # Continuer avec la prochaine itération même en cas d'erreur
        continue
    
    # Vérifier si l'index actuel dépasse un seuil de 10%
    if index >= dix_pourcent * (progression + 1):
        print(f"Avancement: {math.ceil((index / total_lignes) * 100)}% terminé.")
        progression += 1  # Mise à jour de la progression

Erreur lors de l'accès à https://www.xerfi.com/etudes-par-entreprise/UGC-CINE-CITE_347806002: HTTPSConnectionPool(host='www.xerfi.com', port=443): Max retries exceeded with url: /etudes-par-entreprise/UGC-CINE-CITE_347806002 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000013AA4EA7D90>, 'Connection to www.xerfi.com timed out. (connect timeout=10)'))
Erreur lors de l'accès à https://www.xerfi.com/etudes-par-entreprise/UGC-CINE-CITE_347806002: HTTPSConnectionPool(host='www.xerfi.com', port=443): Max retries exceeded with url: /etudes-par-entreprise/UGC-CINE-CITE_347806002 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000013AB5FA8610>, 'Connection to www.xerfi.com timed out. (connect timeout=10)'))
Erreur lors de l'accès à https://www.xerfi.com/etudes-par-entreprise/UGC-CINE-CITE-ILE-DE-FRANCE_395251440: HTTPSConnectionPool(host='www.xerfi.com', port=443): Max retries exceeded with url: /etudes-par-entreprise/UGC-CINE-CI

C:\Users\chris\anaconda3\Lib\site-packages\sumy\summarizers\lsa.py:76: UserWarning: Number of words (6917) is lower than number of sentences (36989). LSA algorithm may not work properly.
  warn(message % (words_count, sentences_count))


Erreur lors de l'accès à https://draaf.occitanie.agriculture.gouv.fr/la-safer-occitanie-r323.html: HTTPSConnectionPool(host='draaf.occitanie.agriculture.gouv.fr', port=443): Max retries exceeded with url: /la-safer-occitanie-r323.html (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))
Erreur lors de l'accès à https://draaf.occitanie.agriculture.gouv.fr/la-safer-occitanie-r323.html: HTTPSConnectionPool(host='draaf.occitanie.agriculture.gouv.fr', port=443): Max retries exceeded with url: /la-safer-occitanie-r323.html (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))
Erreur lors du traitement de la ligne 346: expected name token at '<![� (\x7fi\x06}\x05�Q�a{`\x7f�R'
Erreur lors de l'accès à https://ville-solliestoucas.fr/photo/aauser.php?q=67369471: HTTPSConnect

Erreur lors de l'accès à https://2kbbao1.fbs37.ru/: HTTPSConnectionPool(host='2kbbao1.fbs37.ru', port=443): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000013AB79A3AD0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))


C:\Users\chris\anaconda3\Lib\site-packages\sumy\summarizers\lsa.py:76: UserWarning: Number of words (15) is lower than number of sentences (24). LSA algorithm may not work properly.
  warn(message % (words_count, sentences_count))


Erreur lors de l'accès à https://www.xerfi.com/etudes-par-entreprise/OCP-REPARTITION_388698201: HTTPSConnectionPool(host='www.xerfi.com', port=443): Max retries exceeded with url: /etudes-par-entreprise/OCP-REPARTITION_388698201 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000013AA1D971D0>, 'Connection to www.xerfi.com timed out. (connect timeout=10)'))
Erreur lors de l'accès à https://www.xerfi.com/etudes-par-entreprise/OCP-REPARTITION_388698201: HTTPSConnectionPool(host='www.xerfi.com', port=443): Max retries exceeded with url: /etudes-par-entreprise/OCP-REPARTITION_388698201 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000013AA1D940D0>, 'Connection to www.xerfi.com timed out. (connect timeout=10)'))
Erreur lors du traitement de la ligne 675: 
Erreur lors de l'accès à https://taxidrivers.fr/vtc-aeroport-nice/: HTTPSConnectionPool(host='taxidrivers.fr', port=443): Max retries exceeded with url: /vtc-aeroport-nice/ (

C:\Users\chris\anaconda3\Lib\site-packages\sumy\summarizers\lsa.py:76: UserWarning: Number of words (1) is lower than number of sentences (17). LSA algorithm may not work properly.
  warn(message % (words_count, sentences_count))


Erreur lors de l'accès à https://www.uhaul.com/Locations/: HTTPSConnectionPool(host='www.uhaul.com', port=443): Max retries exceeded with url: /Locations/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000013AA6961550>, 'Connection to www.uhaul.com timed out. (connect timeout=10)'))
Erreur lors de l'accès à https://www.uhaul.com/Locations/: HTTPSConnectionPool(host='www.uhaul.com', port=443): Max retries exceeded with url: /Locations/ (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000013AA582E450>, 'Connection to www.uhaul.com timed out. (connect timeout=10)'))
Erreur lors de l'accès à https://www.boulanger.com/: HTTPSConnectionPool(host='www.boulanger.com', port=443): Read timed out. (read timeout=10)
Erreur lors de l'accès à https://www.boulanger.com/: HTTPSConnectionPool(host='www.boulanger.com', port=443): Read timed out. (read timeout=10)
Erreur lors de l'accès à https://www.pajemploi.urssaf.fr/pajewebinfo/cms/sites

C:\Users\chris\anaconda3\Lib\site-packages\sumy\summarizers\lsa.py:76: UserWarning: Number of words (6946) is lower than number of sentences (72525). LSA algorithm may not work properly.
  warn(message % (words_count, sentences_count))


In [ ]:
# Obtenir la date actuelle
date_actuelle = datetime.now().strftime('%Y-%m-%d')  # Formate la date comme "AAAA-MM-JJ"

# Créer le nom du fichier avec la date
nom_fichier = f'search_Google_{date_actuelle}.csv'

# Exporter le DataFrame en CSV
detail_google_search.to_csv(nom_fichier, index=False)